In [1]:
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
import math
#from .utils.path_to_dataframe import path_to_pandas, pandas_to_local_or_blob
from src.data.dataloader import load_solar_profile
from dateutil.relativedelta import relativedelta

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
from IPython.display import display
import collections
import json

from qbstyles import mpl_style

mpl_style(dark=False)


###  Step 1. Get the solar input and demand data

In [ ]:
# experiment_name = 'June13_Test_Soft_Boundaries_Train_Jan_Test_Jan_094602'

data_path = '/glb/hou/gf.siti/data/rl_sfo/luiz_code/'\
                'solar_fuel_ReinforcementLearning/data/'\
                'solar_profile_hourlyV2.csv'
# path = '/glb/hou/gf.siti/data/rl_sfo/luiz_code'
# results_path = f'{path}/results/{experiment_name}'
# log_path= f'{results_path}/callbacks'

battery_storage_capacity = 5000
curtail_unit_cost = 5
electrolyzer_cost = 0
eval_freq = 500
h2_demand = 64400
h2_price = 5
hyperparameters_optimization = False
initial_month_test = "Jan"
initial_month_train = "Jan"
num_of_months_test = 1
num_of_months_train = 1
penalty_unit_cost = 5
steps_scale_factor = 3000
quantity_of_months_test = 1 

year_df="TMY P90"
year=2015

In [ ]:
whole_year_supply_df = load_solar_profile(data_path=data_path, year=year_df)
first_day_of_the_year = datetime.strptime(f'Jan 01 {year}', '%b %d %Y')
initial_date_test = datetime.strptime(f'{initial_month_test} 01 {year}', '%b %d %Y')
final_date_test = initial_date_test + relativedelta(months=quantity_of_months_test)
initial_date_hours_test = (initial_date_test - first_day_of_the_year).total_seconds() // 3600
final_date_hours_test = (final_date_test - first_day_of_the_year).total_seconds() // 3600
whole_year_supply_df = whole_year_supply_df.iloc[int(initial_date_hours_test):int(final_date_hours_test)]
len_t=len(whole_year_supply_df)

In [ ]:
whole_year_supply_df

In [ ]:
# solar profile scaled to demand 
whole_year_supply_df.plot(x ='hour', y='volume', kind = 'line')  #'scatter')

In [ ]:
whole_year_supply_df.describe()

In [ ]:
# solar profile scaled to demand, the first ten days, so you can see that the 
whole_year_supply_df[:240].plot(x ='hour', y='volume', kind = 'line')  #'scatter')

In [ ]:
whole_year_supply_df.head()

### Step 2  Set up the environment

In [ ]:
len_t  # total number of hours per year

In [ ]:
# define the environment

class State(object):
 
    def __init__(self, T, solar_in, t = 0):
        
        # at solar
        self.solar_in = 0  # solar input level
        self.battery_level = 5000  # stock level
      #  self.electrolyzer_converted_toH2 = 0 # converted H2 level every day
        # H2 
       # self.H2_out = 0  # H2 demand  adding demand to state 
       #self.demand_projection = [0,0,0,0,0,0,0,0,0,0]  #demand_history  # adding demand to state 
         
       # self.H2storage_level = 0.0  # H2 storage stock level
        
        
        self.T = T
        self.t = t



class Action(object):
    def __init__(self):
     #   
        self.solar_to_electrolyzer = 0
        self.solar_to_curtail= 0 
        self.solar_to_battery = 0        
        self.battery_to_electrolyzer = 0
      #  self.electrolyzer_to_H2_out = 0
      #  self.electrolyzer_to_H2storage = 0        
      #  self.H2storage_to_H2_out = 0

class SolarFuelEnvironment(object):
    def __init__(self):
        
        self.T = len_t               # episode duration
  
    #     self.solar_PV_cost = 400     # unit price for solar PV in dollars/kw 
    #     self.battery_cost = 200       # unit cost for battery in dollars/kwh
    #     self.charging_unit_cost = 10
    #     self.discharging_unit_cost = 10
    #     self.curtail_unit_cost = 20   # power wasted as curtail
        
    #  #   self.H2storage_cost = 30     # H2 storage cost
    #     self.electrolyzer_cost = 1000  # /kw H2  ?
        
    #     self.H2_price = 2000  # /kw H2  ?
        
    #     self.electrolyzer_efficiency = 0.5  # /kw H2  ?
        
    #     self.H2_demand = 1150  #   assume it's constant H2 out demand, but this number is per day, to get hourly /24
        
    #     self.battery_storage_capacity =2000  # kw  this scale is like there is no upper limit, can set even higher to imitate that
        
    #  #   self.H2_storage_capacity = 85000 #  
               
    #     self.penalty_unit_cost = self.H2_price  # penalty if not meeting demand, assumption now, higest
    
        self.convert_kw_to_tw = 1e9 # 1 TWh = 1e9 kWh
        self.convert_ton_to_TWh = 5.426e-05 #54.26*1e-6 # 1 H2 Ton = 54.26*1e-6 TWh
        self.t_total = len_t # episode duration
        self.whole_year_supply_df = whole_year_supply_df

        self.battery_unit_cost = 1e-6 #(5/(1*self.convert_ton_to_TWh))/3 #200*self.convert_kw_to_tw #/TWh     # unit cost for battery in dollars/kwh
        self.charging_unit_cost = 1e-6
        self.discharging_unit_cost = 1e-6 #3000
        self.curtail_unit_cost = curtail_unit_cost #5000 #3* 5/(1*self.convert_ton_to_TWh) #20   # power wasted as curtail
        self.H2_demand = h2_demand #TWh 2,600TWh/hr  #1150  # Convert to energy. # MT  assume it's constant H2 out demand, but this scale is per day
        self.battery_storage_capacity = battery_storage_capacity  # Twh  this scale is like there is no upper limit, can set even higher to imitate that ONE DAY OF STORAGE IS GOOD
        self.electrolyzer_cost = electrolyzer_cost #1000*self.convert_kw_to_tw  # /kw H2  ?
        self.H2_price = h2_price #5/(1*self.convert_ton_to_TWh) # /TWh   $5/ton H2 pricve
        self.penalty_unit_cost = penalty_unit_cost #5/(1*self.convert_ton_to_TWh)  # penalty if not meeting demand, assumption now, higest
        
        self.reset()

    def reset(self): #, demand_history_len = 4):

        self.t = 0

    def solar_PV(self,  t): # 
         
        return whole_year_supply_df['volume'].iloc[t]
    def initial_state(self):
        
        return State(self.T,solar_in =0) #, list(self.demand_history))

    def step(self, state, action):
        
        # calculating the reward (profit)
       
        # sales by the H2 sale
        
        revenue = (action.solar_to_electrolyzer+action.battery_to_electrolyzer)* self.H2_price   
        
        
        # solar_prod_cost = self.solar_PV_cost * state.solar_in  # solar cost,don't know should include or not,ignore first
        
        # electrolyzer_cost = (action.solar_to_electrolyzer+action.battery_to_electrolyzer)*self.electrolyzer_cost
       
        battery_cost = state.battery_level*self.battery_unit_cost 
        charging_cost = action.solar_to_battery*self.charging_unit_cost
        discharging_cost = action.battery_to_electrolyzer*self.discharging_unit_cost
        curtail_cost = action.solar_to_curtail*self.curtail_unit_cost
     
        # H2storage_cost = state.H2storage_level*elf.H2storage_cost
        penalty_cost = - self.penalty_unit_cost * (min(action.battery_to_electrolyzer+action.solar_to_electrolyzer-self.H2_demand/24,0))  # needs to check
           
      # electrolyzer_cost = self.electrolyzer_cost*(state.solar_in-action.solar_to_curtail-action.solar_to_battery+action.battery_to_electrolyzer)
   
        
        
       # reward = revenue - solar_prod_cost - H2storage_cost - battery_cost - electrolyzer_cost
       # reward = revenue - solar_prod_cost - battery_cost - electrolyzer_cost
        # assume production is fixed and then just ignore the production cost, can be added later since we are not change production volume for now
        
        reward = revenue - battery_cost - charging_cost - discharging_cost - curtail_cost - penalty_cost
        
        reward_adjusted = reward +  curtail_cost + penalty_cost
        
        

        # calculating the next state
        
      #  self.solar_to_electrolyzer = 0
      #  self.solar_to_curtail= 0 
     #   self.solar_to_battery = 0        
     #   self.battery_to_electrolyzer = 0
       
        
        # the power to H2 converted
        
       # next hour
        
        next_state = State(self.T, self.t)
        
        next_state.solar_in = self.solar_PV(self.t)  # input solar, just read from the dataframe
        
        next_state.battery_level= state.battery_level+action.solar_to_battery-action.battery_to_electrolyzer
        
    #    next_state.electrolyzer_converted_toH2 = self.electrolyzer_efficiency*(state.solar_in-action.solar_to_curtail-action.solar_to_battery+action.battery_to_electrolyzer)
          
       # next_state.battery_level= state.battery_level+action.solar_to_battery-action.battery_to_electrolyzer
        
       # next_state.H2storage_level= state.H2storage_level+ action.electrolyzer_to_H2storage -action.H2storage_to_H2_out
             
        #next_state.factory_stock = min(state.factory_stock + action.factory_stockpile-action.factory_reclaim, self.factory_storage_capacity) 
       
        #next_state.warehouse_stock = min(state.warehouse_stock+action.shippings_to_warehouses-state.demand_history,self.warehouse_storage_capacity)
       # next_state.factory_silo = min(self.production_level+ state.factory_silo-action.factory_stockpile+action.factory_reclaim-action.shippings_to_warehouses,self.factory_silo_capacity)
    
        #next_state.demand_history = self.demand(self.t)
        #next_state.demand_projection = self.demand_future(self.t)
        self.t += 1
       
        

        return next_state, reward, self.t == self.T - 1, reward_adjusted

In [ ]:
#%%  Visualization function 
def prepare_metric_plot(plots_n, n, ylabel):
    plt.subplot(plots_n, 1, n)
    plt.ylabel(ylabel)
    plt.tick_params(axis='x', which='both', bottom=True, top=True, labelbottom=False)

# visualizing one episode - stock levels, production and reorderign actions, and rewards
def visualize_transitions(transitions):
    state_trace, action_trace, reward_trace, reward_adjusted = (transitions.T[0], transitions.T[1], transitions.T[2], transitions.T[3])
    plots_n = 9
    mpl.rcParams['lines.linewidth'] = 2    
    print(f"Return is {sum(reward_trace)}")

    fig = plt.figure(figsize=(15, 30))
    # state plots
    prepare_metric_plot(plots_n, 1, "Input level,\n Solar PV")
    plt.plot(range(len_t), list(map(lambda s: s.solar_in, state_trace)), c='yellow', alpha=0.5)
    
 
    prepare_metric_plot(plots_n, 2, "Battery level")
    plt.plot(range(len_t), list(map(lambda s: s.battery_level, state_trace)), c='purple', alpha=0.5)
    
 
  #  prepare_metric_plot(plots_n, 3, "H2 generated")
  #  plt.plot(range(env.T), list(map(lambda s: s.electrolyzer_converted_toH2, state_trace)), c='orange', alpha=0.5)
    
 
  #  prepare_metric_plot(plots_n, 4, "Demand,\n H2")
  #  plt.plot(range(env.T), list(map(lambda s: s.H2_out, state_trace)), c='red', alpha=0.5)
    
              
    prepare_metric_plot(plots_n, 3, "Curtail,\n Solar")
    plt.plot(range(len_t), list(map(lambda a: a.solar_to_curtail, action_trace)), c='green', alpha=0.5)
    
    prepare_metric_plot(plots_n, 4, "to electrolyzer,\n Solar")
    plt.plot(range(len_t), list(map(lambda a: a.solar_to_electrolyzer, action_trace)), c='cyan', alpha=0.5)
    
    prepare_metric_plot(plots_n, 5, "to battery,\n Solar")
    plt.plot(range(len_t), list(map(lambda a: a.solar_to_battery, action_trace)), c='magenta', alpha=0.5)
    
    prepare_metric_plot(plots_n, 6, "to electrolyzer,\n Battery")
    plt.plot(range(len_t), list(map(lambda a: a.battery_to_electrolyzer, action_trace)), c='purple', alpha=0.5)
 
    
    prepare_metric_plot(plots_n, 7, "Profit")
    plt.plot(range(len_t), reward_trace, c='red', alpha=0.9, linewidth=2)
    
    prepare_metric_plot(plots_n, 7, "Profit")
    plt.plot(range(len_t), reward_adjusted, c='red', alpha=0.9, linewidth=2)

    plt.subplot(plots_n, 1, 8)
    plt.ylabel("Cumulative\nprofit")
   # plt.ylim(0, 10000)
    plt.plot(range(env.T), np.cumsum(reward_adjusted), c='red', alpha=0.9, linewidth=2)
    plt.xlabel("Time step")

In [ ]:
# data frame generation function
def save_transitions(transitions):
    state_trace, action_trace, reward_trace, reward_adjusted = (transitions.T[0], transitions.T[1], transitions.T[2], transitions.T[3])
   
    solar_in=list(map(lambda s: s.solar_in, state_trace))    
    battery_level = list(map(lambda s: s.battery_level, state_trace))
  #  converted_H2= list(map(lambda s: s.electrolyzer_converted_toH2, state_trace))
   # demand_state=list(map(lambda s: s.H2_out, state_trace)) 
    H2_demand = 62402/24  
    solar_to_curtail=list(map(lambda a: a.solar_to_curtail, action_trace))    
    solar_to_electrolyzer=list(map(lambda a: a.solar_to_electrolyzer, action_trace))    
    solar_to_battery=list(map(lambda a: a.solar_to_battery, action_trace))
    battery_to_electrolyzer=list(map(lambda a: a.battery_to_electrolyzer, action_trace))
    profit=reward_trace
    time= range(len_t)
    savedtransition = pd.DataFrame(
    {'time': time,
     'solar_in': solar_in,
     'battery_level': battery_level,
  #   'converted_H2': converted_H2,
     'H2_demand':H2_demand,
     'solar_to_curtail': solar_to_curtail,
     'solar_to_electrolyzer': solar_to_electrolyzer,
     'solar_to_battery': solar_to_battery,
     'battery_to_electrolyzer': battery_to_electrolyzer,
     'profit': profit,
     'profit adjusted': reward_adjusted    
    })
   #  save_transitions.to_csv('/glb/hou/gf.siti/data/rl_sfo/luiz_code/solar_fuel_ReinforcementLearning/data/baseline_1year.csv')
    return savedtransition
   
   

### Step 3  Define policy

In [ ]:
class SQPolicy1(object):
    def __init__(self,threshold_battery): #,factory_shipping_var, factory_reclaim_var, factory_stockpile_var ):
     
       
        self.daily_demand = 64400  # just need the daily electricity demand
        
        self.hourly_demand = 64400/24.0  # just need the daily electricity demand
      
        self.battery_storage_capacity = 5000   #capacity_var
        
        self.battery_threshold = threshold_battery

    def select_action(self, state):
        action = Action()

        solar_in = state.solar_in   #a  use this to represent so it's easier to write the relationship 
        battery_level = state.battery_level # g
       # converted_H2= state.electrolyzer_converted_toH2         
        hourly_demand= self.hourly_demand   # f 
        battery_capacity = self.battery_storage_capacity
        
        battery_threshold =  self.battery_threshold  # adding extra variable to optimize the solution
     #   
         
       # solar_to_curtail= action.solar_to_curtail  # d
       # solar_to_electrolyzer= action.solar_to_electrolyzer  # e
       # solar_to_battery= action.solar_to_battery  #b
       # battery_to_electrolyzer= action.battery_to_electrolyzer # c
    
        # action according to state status and battery level and input and output demand
        
        # simple rules for now
        if solar_in >= hourly_demand:  # can meet the hourly demand, day
                
            if battery_level < battery_threshold:             
                action.solar_to_battery = solar_in-hourly_demand
                action.solar_to_curtail = 0            
                action.battery_to_electrolyzer = 0
                action.solar_to_electrolyzer = hourly_demand
            else:
                action.solar_to_battery = 0
                action.solar_to_curtail = solar_in-hourly_demand            
                action.battery_to_electrolyzer = 0
                action.solar_to_electrolyzer = hourly_demand
            
        else:  # cannot meet the hourly demand from solar in
            
            if battery_level < battery_threshold:             
                action.solar_to_battery = 0
                action.solar_to_curtail = 0            
                action.battery_to_electrolyzer = min(hourly_demand - solar_in, battery_level)
                action.solar_to_electrolyzer = solar_in
            else:
                action.solar_to_battery = 0                           
                action.battery_to_electrolyzer = min(hourly_demand - solar_in, battery_level)
                action.solar_to_electrolyzer = solar_in
                action.solar_to_curtail = 0
            
            
            
            
            
        
   #     if battery_level< battery_threshold:
            
   #         action.solar_to_battery = min(battery_capacity - battery_level,solar_in)
   #         action.solar_to_curtail = 0            
   #         action.battery_to_electrolyzer = 0
   #         action.solar_to_electrolyzer = solar_in- action.solar_to_battery-action.solar_to_curtail
            
  #      else:
            
   #         action.solar_to_battery = 0
   #         action.solar_to_curtail = 0            
   #         action.battery_to_electrolyzer = battery_level-battery_threshold
   #         action.solar_to_electrolyzer = solar_in
            
            
               
            
            
   #     if solar_in >= daily_e_demand: # more supply than demand
   #         if(solar_in-daily_e_demand+battery_level)> battery_capacity:
   #             action.solar_to_curtail = solar_in - daily_e_demand + battery_level-battery_capacity
  #              action.solar_to_battery = battery_capacity - battery_level
  #              action.battery_to_electrolyzer = 0.0
  #              action.solar_to_electrolyzer = daily_e_demand
  #          else:
  #              action.solar_to_curtail = 0.0
  #              action.solar_to_battery = solar_in - daily_e_demand
  #              action.battery_to_electrolyzer = 0.0
  #              action.solar_to_electrolyzer = daily_e_demand
                
  #      else: # not enough
            
  #          if (battery_level + solar_in)> daily_e_demand:
  #              action.battery_to_electrolyzer = daily_e_demand-solar_in
  #              action.solar_to_curtail = 0.0
  #              action.solar_to_electrolyzer = solar_in
  #             # if battery_level<battery_threshold: 
  #              action.solar_to_battery = 0.0
                
   #         else:
   #             action.battery_to_electrolyzer = battery_level
   #             action.solar_to_curtail = 0.0
   #             action.solar_to_electrolyzer = solar_in
               
   #             action.solar_to_battery = 0.0
            
        
        
        return action
         
    
def simulate_episode(env, policy):
    state = env.initial_state()
    transitions = []
    for t in range(env.T):
        action = policy.select_action(state)
        state, reward, _, reward_adjusted = env.step(state, action)
        transitions.append([state, action, reward, reward_adjusted])
    return transitions

# basic policy evaluation process 
def simulate(env, policy, num_episodes):
    returns_trace = []
    for episode in range(num_episodes):
        env.reset()
        returns_trace.append( sum(np.array(simulate_episode(env, policy)).T[2]) )  # sum the rewards for 
    return returns_trace

In [ ]:
env = SolarFuelEnvironment()
policy_sQ = SQPolicy1(200)

In [ ]:
state = env.initial_state()

In [ ]:
state.solar_in

In [ ]:
state.battery_level

In [ ]:
transitions = []

In [ ]:
for t in range(env.T):
    action = policy_sQ.select_action(state)
    state, reward, _, reward_adjusted = env.step(state, action)
    transitions.append([state, action, reward, reward_adjusted])

In [ ]:
print(sum(np.array(transitions).T[2]))

In [ ]:
state_trace, action_trace, reward_trace = (np.array(transitions).T[0],np.array(transitions).T[1],np.array(transitions).T[2])


### Optimizing (s, Q)-policy parameters using Facebook Ax

In [ ]:
from ax import optimize
env = SolarFuelEnvironment()
def func(p):
    policy = SQPolicy1(
        p['threshold_battery'],
       # p['pctthreshold10days_var'],
       # p['pctthreshold7days_var'],
      #  p['pctthreshold3days_var'],
      #  p['pctthreshold1day_var']
    )
    return np.mean(simulate(env, policy, num_episodes = 2))

best_parameters, best_values, experiment, model = optimize(
        parameters=[
       #     { "name": "pctthreshold15days_var",   "type": "range",  "bounds": [0.0, 100000.0], },
       #     { "name": "pctthreshold10days_var",   "type": "range",  "bounds": [0.0, 100000.0], },
      #      { "name": "pctthreshold7days_var",   "type": "range",  "bounds": [0.0, 100000.0], },
      #      { "name": "pctthreshold3days_var",   "type": "range",  "bounds": [0.0, 100000.0], },
            { "name": "threshold_battery",    "type": "range",  "bounds": [200.0, 2000.0], },
        #  { "name": "factory_reclaim",    "type": "range",  "bounds": [0.0, 1650.0], },
        #  { "name": "factory_stockpiling", "type": "range",  "bounds": [0.0, 1650.0], },  
          ],
        evaluation_function=func,
        minimize=False,
        total_trials=50,
    )

In [ ]:
display(best_parameters)
display(best_values)

In [ ]:
means, covariances = best_values
print(means)
print(covariances)

In [ ]:
# evaluating the winning (s,Q) policy more accurately

#env = SolarFuelEnvironment()
#policy_sQ = SQPolicy1(0)

env = SolarFuelEnvironment()
policy_sQ = SQPolicy1(best_parameters['threshold_battery'])

return_trace = simulate(env, policy_sQ, num_episodes = 10)
plt.figure(figsize=(16, 4))
plt.plot(range(len(return_trace)), return_trace)
print(f"Reward: mean {np.mean(return_trace)}, standard deviation {np.std(return_trace)}")


In [ ]:

# tracing the winning (s,Q)  policy
env = SolarFuelEnvironment()
#env = SolarFuelEnvironment()
policy_sQ = SQPolicy1(best_parameters['threshold_battery'])
transitions_sQ = simulate_episode(env, policy_sQ)
visualize_transitions(np.array(transitions_sQ))

In [ ]:
pd_transition=save_transitions(np.array(transitions_sQ))
pd_transition

In [ ]:
pd_transition.to_csv(f'/glb/hou/gf.siti/data/rl_sfo/luiz_code/solar_fuel_ReinforcementLearning/data/baseline_trained_{initial_month_test}_{year_df}.csv')

### Save the actions and states with time to a file

In [ ]:
# merge with whole_year_demand_df, so we know which date to do the actions

# df_concat = pd.concat([whole_year_supply_df, pd_transition], axis=1)

# print(df_concat)

In [ ]:
best_parameters['threshold_battery']